In [5]:
"""
1、告警内容理解。根据输入的告警信息，结合第三方接口数据，判断当前的异常情况（告警对象、异常模式）；
2、分析方法建议。根据当前告警内容，结合应急预案、运维文档和大语言模型自有知识，形成分析方法的建议；
3、分析内容自动提取。根据用户输入的分析内容需求，调用多种第三方接口获取分析数据，并进行总结；
4、处置方法推荐和执行。根据当前上下文的故障场景理解，结合应急预案和第三方接口，形成推荐处置方案，待用户确认后调用第三方接口进行执行。
"""
import json
import os
import random
import dashscope
from dashscope.api_entities.dashscope_response import Role

# 从环境变量获取API Key
api_key = os.environ.get("DASHSCOPE_API_KEY")
if not api_key:
    raise ValueError("请先设置环境变量 DASHSCOPE_API_KEY，再运行本程序！")
dashscope.api_key = api_key

from openai import OpenAI

# 通过第三方接口获取数据库服务器状态
def get_current_status():
    # 生成连接数数据
    connections = random.randint(10, 100)
    # 生成CPU使用率数据
    cpu_usage = round(random.uniform(1, 100), 1)
    # 生成内存使用率数据
    memory_usage = round(random.uniform(10, 100), 1)
    status_info = {
        "连接数": connections,
        "CPU使用率": f"{cpu_usage}%",
        "内存使用率": f"{memory_usage}%"
    }
    return json.dumps(status_info, ensure_ascii=False)

# 封装模型响应函数
def get_response(messages):
    response = dashscope.Generation.call(
        model='qwen-turbo',
        messages=messages,
        tools=tools,
        result_format='message'  # 将输出设置为message形式
    )
    return response
    
current_locals = locals()
current_locals

tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_status",
                "description": "调用监控系统接口，获取当前数据库服务器性能指标，包括：连接数、CPU使用率、内存使用率",
                "parameters": {
                },
                "required": []
            }                
        }
    ]

query = """告警：数据库连接数超过设定阈值
时间：2024-08-03 15:30:00
"""
messages=[
    {"role": "system", "content": "我是运维分析师，用户会告诉我们告警内容。我会基于告警内容，判断当前的异常情况（告警对象、异常模式）"},
    {"role": "user", "content": query}]


In [6]:
while True:
    response = get_response(messages)
    message = response.output.choices[0].message
    messages.append(message)
    #print('response=', response)

    if response.output.choices[0].finish_reason == 'stop':
        break
    
    # 判断用户是否要call function
    if message.tool_calls:
        # 获取fn_name, fn_arguments
        fn_name = message.tool_calls[0]['function']['name']
        fn_arguments = message.tool_calls[0]['function']['arguments']
        arguments_json = json.loads(fn_arguments)
        #print(f'fn_name={fn_name} fn_arguments={fn_arguments}')
        function = current_locals[fn_name]
        tool_response = function(**arguments_json)
        tool_info = {"name": "get_current_weather", "role":"tool", "content": tool_response}
        #print('tool_info=', tool_info)
        messages.append(tool_info)

In [3]:
print(messages)

[{'role': 'system', 'content': '我是运维分析师，用户会告诉我们告警内容。我会基于告警内容，判断当前的异常情况（告警对象、异常模式）'}, {'role': 'user', 'content': '告警：数据库连接数超过设定阈值\n时间：2024-08-03 15:30:00\n'}, Message({'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'get_current_status', 'arguments': '{}'}, 'index': 0, 'id': 'call_70d2441d3fff45dc98c33b', 'type': 'function'}]}), {'name': 'get_current_weather', 'role': 'tool', 'content': '{"连接数": 71, "CPU使用率": "55.3%", "内存使用率": "76.9%"}'}, Message({'role': 'assistant', 'content': '根据当前数据库服务器的性能指标，连接数为71，超过了设定的阈值。这表明可能存在高负载或资源争用的问题。\n\n建议采取以下措施：\n1. **检查应用程序**：确认是否有异常的请求或程序导致连接数激增。\n2. **优化查询**：如果数据库查询效率低下，可能会导致连接数增加。\n3. **调整阈值**：如果连接数是正常情况下的高负载，可以考虑重新评估并调整阈值。\n4. **扩展资源**：如果连接数持续高，可能需要考虑增加服务器资源或进行负载均衡。\n\n请进一步分析具体原因，并采取相应的解决措施。'})]
